In [1]:
import os
import time
import torch
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

import numpy as np
from PIL import Image

from sklearn.metrics import jaccard_score
from medpy.metric.binary import hd, asd 

import ssl

/home/davi/projects/TI0160-visao-computacional/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache()

In [3]:
os.chdir("..")

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context

PATCH_SIZE = (256, 256)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
N_EPOCHS = 30
N_WORKERS = 10
MODELS_PATH = "models/segmentation"

In [5]:
class Covid19Dataset(Dataset):
    def __init__(self, image_paths, mask_paths):
        """
        image_paths: Lista de caminhos das imagens.
        mask_paths: Lista de caminhos das máscaras correspondentes.
        """
        self.transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(PATCH_SIZE),
                torchvision.transforms.ToTensor(),
            ]
        )
        self.image_paths = image_paths
        self.mask_paths = mask_paths


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Carrega a imagem e a máscara
        image = Image.open(img_path).convert('L')
        mask = Image.open(mask_path).convert('L')

        image = self.transform(image)
        mask = self.transform(mask)

        return image, mask


In [6]:
from sklearn.model_selection import train_test_split

root_dir = "data/raw"

all_image_paths = []
all_mask_paths = []

for class_name in os.listdir(root_dir):
    class_dir = os.path.join(root_dir, class_name)

    if os.path.isdir(class_dir):
        image_dir = os.path.join(class_dir, "images")
        mask_dir = os.path.join(class_dir, "masks")

        img_names = sorted(os.listdir(image_dir))

        for img_name in img_names[:250]:
            img_path = os.path.join(image_dir, img_name)
            mask_path = os.path.join(mask_dir, img_name)
            all_image_paths.append(img_path)
            all_mask_paths.append(mask_path)

train_images, test_images, train_masks, test_masks = train_test_split(
    all_image_paths, all_mask_paths, test_size=0.2, random_state=42
)


In [7]:
train_dataset = Covid19Dataset(train_images, train_masks)
test_dataset = Covid19Dataset(test_images, test_masks)

In [8]:
def train(model, loader, loss_fn, optimizer, n_epochs):
    model.train()
    
    for epoch in range(n_epochs):
        running_loss = 0.0

        for images, masks in loader:
            images = images.to(DEVICE)
            masks = masks.to(DEVICE).unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images)
            
            loss = loss_fn(outputs, masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(loader)
        print(f'Epoch {epoch+1}/{n_epochs} / Loss: {epoch_loss:.4f}')

In [9]:
def dice_coefficient(pred, target):
    smooth = 1e-6  # Para evitar divisão por zero
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

def jaccard_index(pred, target):
    pred = pred.flatten()
    target = target.flatten()
    return jaccard_score(target, pred)

def hausdorff_distance(pred, target):
    pred_np = pred.cpu().numpy().astype(bool)
    target_np = target.cpu().numpy().astype(bool)

    return hd(pred_np, target_np)

def average_surface_distance(pred, target):
    pred_np = pred.cpu().numpy().astype(bool)
    target_np = target.cpu().numpy().astype(bool)
    return asd(pred_np, target_np)

def evaluate(model, loader):
    model.eval()
    dice_scores = []
    jaccard_scores = []
    hausdorff_distances = []
    avg_surface_distances = []
    times = []

    with torch.no_grad():
        for images, masks in loader:
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            start_time = time.time()
            outputs = model(images)
            end_time = time.time()

            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()

            for i in range(images.size(0)):
                pred = preds[i].cpu()
                mask = masks[i].cpu()

                dice = dice_coefficient(pred, mask)
                dice_scores.append(dice)

                jaccard = jaccard_index(pred, mask)
                jaccard_scores.append(jaccard)

                hausdorff = hausdorff_distance(pred, mask)
                hausdorff_distances.append(hausdorff)

                avg_surf_dist = average_surface_distance(pred, mask)
                avg_surface_distances.append(avg_surf_dist)

            times.append(end_time - start_time)

    # Média das métricas
    avg_dice = np.mean(dice_scores)
    avg_jaccard = np.mean(jaccard_scores)
    avg_hausdorff = np.mean(hausdorff_distances)
    avg_asd = np.mean(avg_surface_distances)
    total_time = np.mean(times)
    
    return {
        "Mean Dice Coefficient": avg_dice,
        "Mean Jaccard Index": avg_jaccard,
        "Mean Hausdorff Distance": avg_hausdorff,
        "Mean Average Surface Distance": avg_asd,
        "Processing Time (s)": total_time
    }


In [10]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

## Encoder: Densenet201 / Decoder: UNET

In [11]:
preprocess_input = get_preprocessing_fn('densenet201', pretrained='imagenet')

densenet201_unet_model = smp.Unet(
    encoder_name="densenet201",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1,
).to(DEVICE)

optimizer = optim.Adam(densenet201_unet_model.parameters(), lr=1e-4)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE)

model_path = os.path.join(MODELS_PATH, "densenet201_unet_model.pth")

if os.path.exists(model_path):
    densenet201_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado!")
else:
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=True,
        drop_last=False,
        num_workers=N_WORKERS
    )

    train(densenet201_unet_model, train_loader, loss_fn, optimizer, N_EPOCHS)
    save_model(densenet201_unet_model, model_path)
    print("Modelo treinado!")

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=False,
        drop_last=False,
        num_workers=N_WORKERS
)

metrics = evaluate(densenet201_unet_model, test_loader)
print("\nMétricas de avaliação:")
for metric, value in metrics.items():
    print(f"\t{metric}: {value:.4f}")

Epoch 1/30 / Loss: 0.4383
Epoch 2/30 / Loss: 0.2937
Epoch 3/30 / Loss: 0.2255
Epoch 4/30 / Loss: 0.1724
Epoch 5/30 / Loss: 0.1361
Epoch 6/30 / Loss: 0.1101
Epoch 7/30 / Loss: 0.0905
Epoch 8/30 / Loss: 0.0788
Epoch 9/30 / Loss: 0.0663
Epoch 10/30 / Loss: 0.0582
Epoch 11/30 / Loss: 0.0501
Epoch 12/30 / Loss: 0.0454
Epoch 13/30 / Loss: 0.0409
Epoch 14/30 / Loss: 0.0369
Epoch 15/30 / Loss: 0.0331
Epoch 16/30 / Loss: 0.0306
Epoch 17/30 / Loss: 0.0281
Epoch 18/30 / Loss: 0.0265
Epoch 19/30 / Loss: 0.0246
Epoch 20/30 / Loss: 0.0234
Epoch 21/30 / Loss: 0.0223
Epoch 22/30 / Loss: 0.0209
Epoch 23/30 / Loss: 0.0196
Epoch 24/30 / Loss: 0.0187
Epoch 25/30 / Loss: 0.0179
Epoch 26/30 / Loss: 0.0170
Epoch 27/30 / Loss: 0.0160
Epoch 28/30 / Loss: 0.0154
Epoch 29/30 / Loss: 0.0146
Epoch 30/30 / Loss: 0.0146
Modelo treinado!

Métricas de avaliação:
	Mean Dice Coefficient: 0.9815
	Mean Jaccard Index: 0.9640
	Mean Hausdorff Distance: 7.0597
	Mean Average Surface Distance: 0.0538
	Processing Time (s): 0.406

## Encoder: Resnet152 / Decoder: UNET

In [16]:
preprocess_input = get_preprocessing_fn("resnet152", pretrained="imagenet")

resnet152_unet_model = smp.Unet(
    encoder_name="resnet152",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1,
).to(DEVICE)

optimizer = optim.Adam(resnet152_unet_model.parameters(), lr=1e-4)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE)

model_path = os.path.join(MODELS_PATH, "resnet152_unet_model.pth")

if os.path.exists(model_path):
    resnet152_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado!")
else:
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=True,
        drop_last=False,
        num_workers=N_WORKERS
    )

    train(resnet152_unet_model, train_loader, loss_fn, optimizer, N_EPOCHS)
    save_model(resnet152_unet_model, model_path)
    print("Modelo treinado!")

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=False,
        drop_last=False,
        num_workers=N_WORKERS
)

metrics = evaluate(resnet152_unet_model, test_loader)
print("\nMétricas de avaliação:")
for metric, value in metrics.items():
    print(f"\t{metric}: {value:.4f}")

/tmp/ipykernel_770/3369070554.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet152_unet_model.load_state_dict(torch.load(model_path))


Modelo carregado!

Métricas de avaliação:
	Mean Dice Coefficient: 0.9830
	Mean Jaccard Index: 0.9669
	Mean Hausdorff Distance: 5.6518
	Mean Average Surface Distance: 0.0348
	Processing Time (s): 0.0251


## Encoder: MobileNetV2 / Decoder: UNET

In [21]:
preprocess_input = get_preprocessing_fn("mobilenet_v2", pretrained="imagenet")

mobilenet_v2_unet_model = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1,
).to(DEVICE)

optimizer = optim.Adam(mobilenet_v2_unet_model.parameters(), lr=1e-4)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE)

model_path = os.path.join(MODELS_PATH, "mobilenet_v2_unet_model.pth")

if os.path.exists(model_path):
    mobilenet_v2_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado!")
else:
    print("Treinando modelo...")
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=True,
        drop_last=False,
        num_workers=N_WORKERS
    )

    train(mobilenet_v2_unet_model, train_loader, loss_fn, optimizer, N_EPOCHS)
    save_model(mobilenet_v2_unet_model, model_path)
    print("Modelo treinado!")

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=False,
        drop_last=False,
        num_workers=N_WORKERS
)

metrics = evaluate(mobilenet_v2_unet_model, test_loader)
print("\nMétricas de avaliação:")
for metric, value in metrics.items():
    print(f"\t{metric}: {value:.4f}")

Modelo carregado!


/tmp/ipykernel_770/2516534121.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mobilenet_v2_unet_model.load_state_dict(torch.load(model_path))



Métricas de avaliação:
	Mean Dice Coefficient: 0.9764
	Mean Jaccard Index: 0.9542
	Mean Hausdorff Distance: 10.2416
	Mean Average Surface Distance: 0.1046
	Processing Time (s): 0.0067


## Encoder: Xception / Decoder: UNET

In [25]:
preprocess_input = get_preprocessing_fn("xception", pretrained="imagenet")

xception_unet_model = smp.Unet(
    encoder_name="xception",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1,
).to(DEVICE)

optimizer = optim.Adam(xception_unet_model.parameters(), lr=1e-4)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE)

model_path = os.path.join(MODELS_PATH, "xception_unet_model.pth")

if os.path.exists(model_path):
    xception_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado!")
else:
    print("Treinando modelo...")
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=True,
        drop_last=False,
        num_workers=N_WORKERS
    )

    train(xception_unet_model, train_loader, loss_fn, optimizer, N_EPOCHS)
    save_model(xception_unet_model, model_path)
    print("Modelo treinado!")

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=False,
        drop_last=False,
        num_workers=N_WORKERS
)

metrics = evaluate(xception_unet_model, test_loader)
print("\nMétricas de avaliação:")
for metric, value in metrics.items():
    print(f"\t{metric}: {value:.4f}")

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /home/davi/.cache/torch/hub/checkpoints/xception-43020ad28.pth
100%|██████████| 87.4M/87.4M [04:17<00:00, 356kB/s]


Treinando modelo...
Epoch 1/30 / Loss: 0.4070
Epoch 2/30 / Loss: 0.2316
Epoch 3/30 / Loss: 0.1620
Epoch 4/30 / Loss: 0.1225
Epoch 5/30 / Loss: 0.0986
Epoch 6/30 / Loss: 0.0803
Epoch 7/30 / Loss: 0.0676
Epoch 8/30 / Loss: 0.0576
Epoch 9/30 / Loss: 0.0505
Epoch 10/30 / Loss: 0.0445
Epoch 11/30 / Loss: 0.0401
Epoch 12/30 / Loss: 0.0361
Epoch 13/30 / Loss: 0.0329
Epoch 14/30 / Loss: 0.0299
Epoch 15/30 / Loss: 0.0280
Epoch 16/30 / Loss: 0.0257
Epoch 17/30 / Loss: 0.0238
Epoch 18/30 / Loss: 0.0232
Epoch 19/30 / Loss: 0.0221
Epoch 20/30 / Loss: 0.0209
Epoch 21/30 / Loss: 0.0191
Epoch 22/30 / Loss: 0.0187
Epoch 23/30 / Loss: 0.0181
Epoch 24/30 / Loss: 0.0170
Epoch 25/30 / Loss: 0.0158
Epoch 26/30 / Loss: 0.0152
Epoch 27/30 / Loss: 0.0149
Epoch 28/30 / Loss: 0.0143
Epoch 29/30 / Loss: 0.0137
Epoch 30/30 / Loss: 0.0132
Modelo treinado!

Métricas de avaliação:
	Mean Dice Coefficient: 0.9794
	Mean Jaccard Index: 0.9601
	Mean Hausdorff Distance: 8.9616
	Mean Average Surface Distance: 0.0950
	Proces

## Encoder: VGG19 / Decoder: UNET

In [26]:
preprocess_input = get_preprocessing_fn("vgg19", pretrained="imagenet")

vgg19_unet_model = smp.Unet(
    encoder_name="vgg19",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1,
).to(DEVICE)

optimizer = optim.Adam(vgg19_unet_model.parameters(), lr=1e-4)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE)

model_path = os.path.join(MODELS_PATH, "vgg19_unet_model.pth")

if os.path.exists(model_path):
    vgg19_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado!")
else:
    print("Treinando modelo...")
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=True,
        drop_last=False,
        num_workers=N_WORKERS
    )

    train(vgg19_unet_model, train_loader, loss_fn, optimizer, N_EPOCHS)
    save_model(vgg19_unet_model, model_path)
    print("Modelo treinado!")

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=False,
        drop_last=False,
        num_workers=N_WORKERS
)

metrics = evaluate(vgg19_unet_model, test_loader)
print("\nMétricas de avaliação:")
for metric, value in metrics.items():
    print(f"\t{metric}: {value:.4f}")

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /home/davi/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:11<00:00, 51.4MB/s] 


Treinando modelo...
Epoch 1/30 / Loss: 0.4269
Epoch 2/30 / Loss: 0.2784
Epoch 3/30 / Loss: 0.2206
Epoch 4/30 / Loss: 0.1808
Epoch 5/30 / Loss: 0.1519
Epoch 6/30 / Loss: 0.1290
Epoch 7/30 / Loss: 0.1107
Epoch 8/30 / Loss: 0.0950
Epoch 9/30 / Loss: 0.0832
Epoch 10/30 / Loss: 0.0736
Epoch 11/30 / Loss: 0.0651
Epoch 12/30 / Loss: 0.0586
Epoch 13/30 / Loss: 0.0524
Epoch 14/30 / Loss: 0.0472
Epoch 15/30 / Loss: 0.0426
Epoch 16/30 / Loss: 0.0394
Epoch 17/30 / Loss: 0.0361
Epoch 18/30 / Loss: 0.0336
Epoch 19/30 / Loss: 0.0321
Epoch 20/30 / Loss: 0.0287
Epoch 21/30 / Loss: 0.0275
Epoch 22/30 / Loss: 0.0256
Epoch 23/30 / Loss: 0.0236
Epoch 24/30 / Loss: 0.0224
Epoch 25/30 / Loss: 0.0212
Epoch 26/30 / Loss: 0.0197
Epoch 27/30 / Loss: 0.0186
Epoch 28/30 / Loss: 0.0180
Epoch 29/30 / Loss: 0.0170
Epoch 30/30 / Loss: 0.0160
Modelo treinado!

Métricas de avaliação:
	Mean Dice Coefficient: 0.9840
	Mean Jaccard Index: 0.9688
	Mean Hausdorff Distance: 6.9957
	Mean Average Surface Distance: 0.0483
	Proces

## Encoder: InceptionV4 / Decoder: UNET

In [12]:
preprocess_input = get_preprocessing_fn("inceptionv4", pretrained="imagenet+background")

inceptionv4_unet_model = smp.Unet(
    encoder_name="inceptionv4",
    encoder_weights="imagenet+background",
    in_channels=1,
    classes=1,
).to(DEVICE)

optimizer = optim.Adam(inceptionv4_unet_model.parameters(), lr=1e-4)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE)

model_path = os.path.join(MODELS_PATH, "inceptionv4_unet_model.pth")

if os.path.exists(model_path):
    inceptionv4_unet_model.load_state_dict(torch.load(model_path))
    print("Modelo carregado!")
else:
    print("Treinando modelo...")
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=True,
        drop_last=False,
        num_workers=N_WORKERS
    )

    train(inceptionv4_unet_model, train_loader, loss_fn, optimizer, N_EPOCHS)
    save_model(inceptionv4_unet_model, model_path)
    print("Modelo treinado!")

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        pin_memory = True,
        shuffle=False,
        drop_last=False,
        num_workers=N_WORKERS
)

metrics = evaluate(inceptionv4_unet_model, test_loader)
print("\nMétricas de avaliação:")
for metric, value in metrics.items():
    print(f"\t{metric}: {value:.4f}")

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/inceptionv4-8e4777a0.pth" to /home/davi/.cache/torch/hub/checkpoints/inceptionv4-8e4777a0.pth
100%|██████████| 163M/163M [08:00<00:00, 356kB/s] 


Treinando modelo...
Epoch 1/30 / Loss: 0.3883
Epoch 2/30 / Loss: 0.1879
Epoch 3/30 / Loss: 0.1314
Epoch 4/30 / Loss: 0.1019
Epoch 5/30 / Loss: 0.0839
Epoch 6/30 / Loss: 0.0696
Epoch 7/30 / Loss: 0.0596
Epoch 8/30 / Loss: 0.0518
Epoch 9/30 / Loss: 0.0452
Epoch 10/30 / Loss: 0.0404
Epoch 11/30 / Loss: 0.0361
Epoch 12/30 / Loss: 0.0328
Epoch 13/30 / Loss: 0.0305
Epoch 14/30 / Loss: 0.0279
Epoch 15/30 / Loss: 0.0254
Epoch 16/30 / Loss: 0.0237
Epoch 17/30 / Loss: 0.0226
Epoch 18/30 / Loss: 0.0211
Epoch 19/30 / Loss: 0.0199
Epoch 20/30 / Loss: 0.0191
Epoch 21/30 / Loss: 0.0177
Epoch 22/30 / Loss: 0.0169
Epoch 23/30 / Loss: 0.0175
Epoch 24/30 / Loss: 0.0161
Epoch 25/30 / Loss: 0.0151
Epoch 26/30 / Loss: 0.0145
Epoch 27/30 / Loss: 0.0136
Epoch 28/30 / Loss: 0.0133
Epoch 29/30 / Loss: 0.0124
Epoch 30/30 / Loss: 0.0117
Modelo treinado!

Métricas de avaliação:
	Mean Dice Coefficient: 0.9812
	Mean Jaccard Index: 0.9634
	Mean Hausdorff Distance: 7.7922
	Mean Average Surface Distance: 0.0748
	Proces